In [1]:
## Import packages
import numpy as np
import math as m

In [2]:
## Control parameters
M = 8 # Lattice size
M_2 = M*M

T_therm = 10 # Termalization steps
T_values = 100 # Amount of data points to take
interval = 5 # Amount of MC−steps between two saved configurations

betas = np.arange(0, 1, 0.1) # Different inv. temperatures to evaluate

In [3]:
## Nearest neigbour implementation
neigbours = np.array([[(i//M)*M+(i+1)%M, (i+M)%M_2, (i//M)*M+(i-1)%M, (i-M)%M_2] for i in np.arange(M_2)])

In [4]:
## Wolff−Cluster algorithm
def sweep_WC(lattice, p):
    i = np.random.randint(0, M_2)
    C, Pocket = [i], [i] # C is the cluster of spins which is being flipped
    while Pocket:
        for nbr in neigbours[Pocket[0]]: # "neigbours" variable contains every neigbour of each spin
            if nbr not in C:
                if lattice[nbr] == lattice[Pocket[0]]:
                    if np.random.random() < p:
                        Pocket.append(nbr)
                        C.append(nbr)
        del Pocket[0]
    lattice[C] = -lattice[C]
    return lattice

In [5]:
## Actual simulation
for j, beta in enumerate(betas):
    lattice = np.random.choice([-1, 1], M_2)
    configurations = np.zeros([T_values, M_2])
    
    p = 1 - m.exp(-2.0*beta)
    
    # Thermalization
    for t in np.arange(T_therm):
        lattice = sweep_WC(lattice, p)
    
    # Eliminate Autocorrelation
    configurations[0, :] = lattice[:]
    for i in np.arange(1, T_values):
        for n in np.arange(interval):
            lattice = sweep_WC(lattice, p)
        
        configurations[i, :] = lattice[:]
    
    # Save configurations for specific inv. temperature
    np.save('beta_' + str(int(10*beta)), configurations)
    print("Saved configurations for beta = " + str(np.round(beta, 1)))

Saved configurations for beta = 0.0
Saved configurations for beta = 0.1
Saved configurations for beta = 0.2
Saved configurations for beta = 0.3
Saved configurations for beta = 0.4
Saved configurations for beta = 0.5
Saved configurations for beta = 0.6
Saved configurations for beta = 0.7
Saved configurations for beta = 0.8
Saved configurations for beta = 0.9
